In [ ]:
%reload_kedro

In [ ]:
from crypto_thesis.utils import build_log_return
import matplotlib.pyplot as plt

In [ ]:
target_name = catalog.load("params:spine_preprocessing.target_name")
df_prm = catalog.load("prm_binance")

### Remarks on time series modeling

In [ ]:
df_prm_filtered = df_prm[df_prm["symbol"] == target_name]
assert df_prm_filtered["symbol"].nunique() == 1, "More than 1 target selected, review."
df_prm_filtered = df_prm_filtered[["open_time", "close"]]

In [ ]:
df_prm_filtered = df_prm[df_prm["symbol"] == target_name]
assert df_prm_filtered["symbol"].nunique() == 1, "More than 1 target selected, review."
df_prm_filtered = df_prm_filtered[["open_time", "close"]]

df_prm_filtered_logret = build_log_return(df=df_prm_filtered)
df_prm_filtered_logret.loc[:, "log_return_shift"] = df_prm_filtered_logret["log_return"].shift()

df_prm_filtered_logret.loc[:, "pctchg"] = df_prm_filtered_logret["close"].pct_change()
df_prm_filtered_logret.loc[:, "pctchg_shift"] = df_prm_filtered_logret["pctchg"].shift()

df_prm_filtered_dropped = df_prm_filtered_logret.dropna() #remove first 2 data points
assert df_prm_filtered_dropped.shape[0] == df_prm_filtered_logret.shape[0] - 2

In [ ]:
df_px = df_prm_filtered_dropped.set_index("open_time").sort_index()[["close", "shift"]]
df_logret = df_prm_filtered_dropped.set_index("open_time").sort_index()[["log_return", "log_return_shift"]]
df_pctchg = df_prm_filtered_dropped.set_index("open_time").sort_index()[["pctchg", "pctchg_shift"]]

In [ ]:
plt.scatter(df_px["shift"], df_px["close"])
plt.xlabel(f"Close price of {target_name} (t)")
plt.ylabel(f"Close price of {target_name} (t+1)")
plt.savefig("/Users/gpalazzo/Desktop/fig_closePx_target_scatter_t_t1.png")

In [ ]:
plt.scatter(df_logret["log_return_shift"], df_logret["log_return"])
plt.xlabel(f"Log-returns of {target_name} (t)")
plt.ylabel(f"Log-returns of {target_name} (t+1)")
plt.savefig("/Users/gpalazzo/Desktop/fig_logret_target_scatter_t_t1.png")

In [ ]:
plt.scatter(df_pctchg["pctchg_shift"], df_pctchg["pctchg"])
plt.xlabel(f"Return of {target_name} (t)")
plt.ylabel(f"Return of {target_name} (t+1)")
plt.savefig("/Users/gpalazzo/Desktop/fig_pctchg_target_scatter_t_t1.png")